In [1]:
import numpy as np

def calc_posterior(mi, prior=0.5, prec = 100000):
    test_vals = [x / prec for x in range(1, prec)]
    max_t = None
    for t in test_vals:
        if t*np.log(t/prior)+(1-t)*np.log((1-t)/(1-prior)) <= mi:
            if  max_t is None or t > max_t:
                max_t = t
    return max_t

def dp_epsilon_to_posterior_success(epsilon):
    return 1 - 1./(1+np.exp(epsilon))

def dp_ps_to_epsilon(ps):
    return np.log(ps / (1-ps))

In [2]:
mi_to_eps = {}
for mi in [1/128, 1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1.0, 2.0, 4.0]:
    ps = calc_posterior(mi)
    mi_to_eps[mi] = dp_ps_to_epsilon(ps)

In [3]:
q4_results = {"1-URGENT": 999,
"2-HIGH": 997,
"3-MEDIUM": 1031,
"4-NOT SPECIFIED": 989,
"5-LOW": 1077}

In [4]:
results = {}
mi_range = [1/128, 1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1.0, 2.0, 4.0]
for key in q4_results:
    results[key] = []
    true_count = q4_results[key]
    for mi in mi_range:
        eps = mi_to_eps[mi]
        count_results = []
        for _ in range(1000):
            scale = 5. / eps  # Scale parameter for Laplace noise
            noise = np.random.laplace(loc=0, scale=scale)
            count_result = true_count + noise
            count_results.append(count_result)
        rel_error = np.average(
                        [100*abs(
                            count_results[i] - true_count
                            ) / true_count for i in range(len(count_results))]
                    )
        results[key].append(rel_error)

In [5]:
results

{'1-URGENT': [1.9094966190370435,
  1.4390167527426396,
  0.9905998292811071,
  0.6914595858610769,
  0.46419855560261797,
  0.31079773045000797,
  0.17593472263054316,
  0.04416950355088001,
  0.04375585811854141,
  0.042652271600614695],
 '2-HIGH': [2.0380113214257403,
  1.4006948453598298,
  1.0362043387791677,
  0.6815240241527991,
  0.46012590791650193,
  0.31987457214214776,
  0.16049147051813964,
  0.04476538026303727,
  0.04502451991323763,
  0.04347198844732975],
 '3-MEDIUM': [2.015067079989976,
  1.3656586075759645,
  0.985136793056168,
  0.6680317514185643,
  0.44201784625012314,
  0.29375011856122263,
  0.16316300840274026,
  0.0427369536474568,
  0.041347286714669786,
  0.040849264576639496],
 '4-NOT SPECIFIED': [2.0639185449041406,
  1.384206214467273,
  1.064612070607331,
  0.6884686281536756,
  0.4840810775535314,
  0.3230233935064605,
  0.1731452507848902,
  0.04499361879423069,
  0.041972392832094134,
  0.04308354963735532],
 '5-LOW': [1.7801016596146355,
  1.29068435